In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date, timedelta

In [2]:
df = pd.read_csv('../Datasets/Covid/covid_19_clean_complete.csv')

In [3]:
print(df.info())  ##General info for dataset
df[df['Country/Region']=='Greece'].tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13932 entries, 0 to 13931
Data columns (total 8 columns):
Province/State    6426 non-null object
Country/Region    13932 non-null object
Lat               13932 non-null float64
Long              13932 non-null float64
Date              13932 non-null object
Confirmed         13932 non-null int64
Deaths            13932 non-null int64
Recovered         13932 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 870.9+ KB
None


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
12679,NaN,Greece,39.0742,21.8243,3/11/20,99,1,0
12937,NaN,Greece,39.0742,21.8243,3/12/20,99,1,0
13195,NaN,Greece,39.0742,21.8243,3/13/20,190,1,0
13453,NaN,Greece,39.0742,21.8243,3/14/20,228,3,8
13711,NaN,Greece,39.0742,21.8243,3/15/20,331,4,8


In [4]:
df['Day']=df['Date'].apply(lambda x: int(x.split('/')[1]))           ##Turn Date into
df['Month']=df['Date'].apply(lambda x: int(x.split('/')[0]))         ##usefull, countable
df.drop(['Date'],axis=1,inplace=True)                                ##data.

In [7]:
Country_List = ['Italy','Greece','Albania','Bulgaria','North Macedonia','Turkey','Egypt'] ##Countries to be listed
dt=2 ##1 For last day, 2 for the day before the last one

for i in Country_List:
    print('Confirmed in '+ i +': ' + str(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Confirmed'].to_numpy()[0])+'\nConfirmed only yesterday: '+str(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Confirmed'].to_numpy()[0]-df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt+1)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt+1)).split('-')[2])]['Confirmed'].to_numpy()[0])+'\nDeaths in ' + i + ': ' + str(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Deaths'].to_numpy()[0]))
    print('First Confirmed: ' + str(df[df['Country/Region']==i][df['Confirmed']>0][df['Month']==df[df['Country/Region']==i][df['Confirmed']>0]['Month'].min()]['Day'].min())+ '/'+ str(df[df['Country/Region']==i][df['Confirmed']>0]['Month'].min()) + '\n')

print('As of '+str(date.today()-timedelta(days=dt))+'.')

Confirmed in Italy: 24747
Confirmed only yesterday: 3590
Deaths in Italy: 1809
First Confirmed: 31/1

Confirmed in Greece: 331
Confirmed only yesterday: 103
Deaths in Greece: 4
First Confirmed: 26/2

Confirmed in Albania: 42
Confirmed only yesterday: 4
Deaths in Albania: 1
First Confirmed: 9/3

Confirmed in Bulgaria: 51
Confirmed only yesterday: 10
Deaths in Bulgaria: 2
First Confirmed: 8/3

Confirmed in North Macedonia: 14
Confirmed only yesterday: 0
Deaths in North Macedonia: 0
First Confirmed: 26/2

Confirmed in Turkey: 6
Confirmed only yesterday: 1
Deaths in Turkey: 0
First Confirmed: 11/3

Confirmed in Egypt: 110
Confirmed only yesterday: 1
Deaths in Egypt: 2
First Confirmed: 14/2

As of 2020-03-15.


c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
